In [1]:
# Import modules
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl.preprocess import FAST
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.freesurfer import FSCommand, MRIConvert

# Set up study specific variables
#project_home = '/Volumes/iang/active/ELS/ELS_FreeSurfer/Analysis'
project_home = '/Users/lucindasisk/Dropbox/fs_practice'

#fs_subjdir = '/Volumes/iang/active/ELS/ELS_FreeSurfer/ELS_FS_subjDir'
fs_subjdir = '/Users/lucindasisk/Dropbox/fs_practice/ELS_FS_subjDir'

workflow_dir = project_home + '/workflows'
subj_proc = project_home + '/proc/subject'
group_proc = project_home + '/proc/group'
template_proc = project_home + '/proc/template'
subject_info = project_home + '/misc/subjects.csv' 
template_sub = ['011-T1']

#set default FreeSurfer subjects dir
FSCommand.set_default_subjects_dir(fs_subjdir)

In [2]:
######### File handling #########

#Pass in list to freesurfer source node (subs) 
fs_source = MapNode(FreeSurferSource(subjects_dir = fs_subjdir), 
                    name = 'fs_source', iterfield = ['subject_id'])
fs_source.inputs.subject_id = template_sub

#set up datasink
datasink = Node(DataSink(base_directory = template_proc),
                name = 'datasink')

In [ ]:
######### Template creation functions #########
def makeTemplate(dimensions, num_proc):
    output_name = 'ELS_CT_'
    
    call(['antsMultivariateTemplateConstruction2.sh –d 3 –o %s –b 1 –r 1 –c 2 –j %d *_anat.nii' % (output_name, num_proc)])
    
    return(template)
cmd = 'antsMultivariateTemplateConstruction2.sh –d 3 –o ELS_CT_ –b 1 –r 1 –c 2 –j 6 *_anat.nii'


In [3]:
######### Template creation nodes #########

#convert freesurfer files to .nii

convert_to_nii = MapNode(MRIConvert(out_file='brainmask.nii.gz',
                                    out_type='niigz'), 
                         name='convert_to_nii', 
                         iterfield = ['in_file'])

#reorient files for safety :)
reorient_to_std = MapNode(Reorient2Std(),
                         name = 'reorient_to_std',
                         iterfield = ['in_file'])

#pass files into template function (normalized, pre-skull-stripping) --> T1.mgz

#create tissue and brain priors

#pull tissue and brainmask.mgz's for each sub and convert to .nii

#take converted .nii's and reorient

#resulting brainmask gets registered to template (1) ---> results in registration matrix, "warpfield"

#brainmask gets run through segmentation (2) ---> results in segmentation into 4 tissue classes (fslfast)
segment = MapNode(FAST(number_classes = 4), name = 'segment', iterfield = ['in_files'])
#need CSF, cortical gray matter, subcortical gray matter, white matter

#brainmask gets turned into *actual* brainmask (3) ---> binarized brain mask

#apply transformation from registered brainmask to binarized brain mask + tissue segmentations ---> applytransform (FS or FSL)

#average binarized brain mask across all subjects

#average each of the tissue classes together



In [4]:
######### Template and tissue segmentation workflow #########
template_tissue_flow = Workflow(name = "template_tissue_flow")
template_tissue_flow.connect([(fs_source, convert_to_nii, [('brainmask','in_file')]),
                              (convert_to_nii, reorient_to_std, [('out_file', 'in_file')]),
                              (reorient_to_std, segment, [('out_file', 'in_files')]),
                              (segment, datasink, [('tissue_class_files', 'tissue_class_files')])
                             ])

template_tissue_flow.base_dir = workflow_dir
template_tissue_flow.write_graph(graph2use = 'flat')
template_tissue_flow.run('MultiProc', plugin_args={'n_procs': 1})

OSError: command 'dot' could not be found on host LSmacbook.local
Interface CommandLine failed to run. 

In [ ]:
######### Cortical thickness functions #########

cmd = 'antsCorticalThickness.sh -d 3 -a t1.nii.gz -e Template.nii.gz -m brainmask.nii.gz -p segmentationPriors%d.nii.gz -o subject'

In [ ]:
######### Cortical thickness nodes #########


In [ ]:
######### Cortical thickness workflow #########